# 1. Import the necessary libraries for you to be able to query 4square API

In [12]:
import requests
import json
import pandas as pd
from getpass import getpass
import folium
import os
from dotenv import load_dotenv
load_dotenv() # load_env
token = os.getenv("token")

In [13]:
url = "https://api.foursquare.com/v3/places/search"

headers = {
    "accept": "application/json",
    "Authorization": token
}

response = requests.get(url, headers=headers)

print(response.text)

{"results":[{"fsq_id":"5c4714533ba767002caedd2b","categories":[{"id":13302,"name":"Mediterranean Restaurant","short_name":"Mediterranean","plural_name":"Mediterranean Restaurants","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/mediterranean_","suffix":".png"}}],"chains":[],"closed_bucket":"VeryLikelyOpen","distance":179,"geocodes":{"drop_off":{"latitude":41.387677,"longitude":2.168101},"main":{"latitude":41.387526,"longitude":2.167989},"roof":{"latitude":41.387526,"longitude":2.167989}},"link":"/v3/places/5c4714533ba767002caedd2b","location":{"address":"Rambla Catalunya, 3","admin_region":"Cataluña","country":"ES","cross_street":"","formatted_address":"Rambla Catalunya, 3, 08007 Barcelona Catalunya","locality":"Barcelona","postcode":"08007","region":"Catalunya"},"name":"Honest Greens Rambla Catalunya","related_places":{},"timezone":"Europe/Madrid"},{"fsq_id":"4b71cd74f964a5204b5d2de3","categories":[{"id":17018,"name":"Bookstore","short_name":"Bookstore","plural_name":"Bo

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [71]:
lat_ih = 41.3977461
lon_ih = 2.187674417

def requests_for_foursquare (query, lat, lon, radius=500, limit=1):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")


res = requests_for_foursquare("coffee", lat_ih, lon_ih, limit = 10)


names = [(res["results"][i]["name"]) for i in range(10)]
location = [(res["results"][i]["location"]["address"]) for i in range(10)]
new_dict = {}
new_dict["name"] = names
new_dict["location"] = location
df = pd.DataFrame(new_dict)
df

,name,location
0,SKYE Coffee Co,"Calle Pamplona, 88"
1,Three Marks Coffee,"Ausiàs Marc, 151"
2,El Cafè Blau,"Casp, 192"
3,El Bar,"Ribes, 46"
4,Eva-2,"Bolivia, 21"
5,Granja Mabel,"Marina, 114-116"
6,Espacio 88,"Carrer Pamplona, 88"
7,Cantina Restobar,"Marina, 164"
8,De Pa i Xocolata,"Carrer de la Marina, 92"
9,La Rourera,"Gran Vía de Les Corts Catalanes, 792"


# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [48]:
res = requests_for_foursquare("coffee", lat_ih, lon_ih, limit = 20)

names = [(res["results"][i]["name"]) for i in range(5)]
location = [(res["results"][i]["location"]["address"]) for i in range(5)]
new_dict = {}
new_dict["name"] = names
new_dict["location"] = location
closedf = pd.DataFrame(new_dict)
closedf

,name,location
0,SKYE Coffee Co,"Calle Pamplona, 88"
1,Three Marks Coffee,"Ausiàs Marc, 151"
2,El Cafè Blau,"Casp, 192"
3,El Bar,"Ribes, 46"
4,Eva-2,"Bolivia, 21"


# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [76]:
def foursquare_places (venue):

    endpoint = f'https://api.foursquare.com/v3/places/search?query={venue}'
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    try:
        response = requests.get(endpoint, headers=headers)
        data = response.json()
        data = data['results']
        results = []
        for i in data:
            name = i['name']
            location = i['geocodes']['main']
            lat = location.get('latitude')
            lng = location.get('longitude')
            results.append({
                'name': name,
                'lat': lat,
                'lng': lng
            })
        return results
    except:
        print("Error:")
        return []


foursquare_places("restaurant")

[{'name': 'Honest Greens Rambla Catalunya', 'lat': 41.387526, 'lng': 2.167989},
 {'name': 'Ciutat Comtal', 'lat': 41.388858, 'lng': 2.166857},
 {'name': "Chivuo's", 'lat': 41.382918, 'lng': 2.170022},
 {'name': 'Koy Shunka', 'lat': 41.385841, 'lng': 2.175345},
 {'name': 'Vinitus', 'lat': 41.389977, 'lng': 2.164035},
 {'name': 'Casa Lolea', 'lat': 41.388833, 'lng': 2.177238},
 {'name': 'La Alcoba Azul', 'lat': 41.382897, 'lng': 2.175509},
 {'name': 'La Flauta', 'lat': 41.386876, 'lng': 2.161108},
 {'name': 'Reserva Ibérica', 'lat': 41.390783, 'lng': 2.163465},
 {'name': 'Funky Bakers', 'lat': 41.394916, 'lng': 2.172866}]

# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [80]:
! mkdir data
df.to_json("data/coffee_shops.json", index=False)

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [ ]:
# your code here